# D06 Thống kê cơ bản và `groupby()`

## Mục đích

Giới thiệu các hàm thống kê trên Pandas và thống kê phân tầng bằng `groupby()`.


## Thống kê bằng Pandas

Pandas hỗ trợ sẵn một số hàm thống kê đơn giản như trung bình hay trung vị. Bạn có thể thống kê cho một cột (dữ liệu dạng series):

In [1]:
import pandas as pd

d = pd.read_excel("hrm.xlsx", index_col="id")
d["weight"].mean()

55.68827160493827

Hoặc cho nhiều cột (dữ liệu dạng data frame):

In [2]:
d[["weight", "height"]].mean()

weight    55.688272
height     1.595127
dtype: float64

Khi thống kê trên data frame, bạn có thể thống kê trên từng dòng thay vì theo cột.

In [3]:
d.filter(like="q_fssg_").mean(axis=1)

id
223     0.833333
236     0.916667
256     1.416667
296     1.333333
310     1.083333
          ...   
4200    1.500000
4214    0.583333
4216    0.916667
4220    0.583333
4240    0.333333
Length: 330, dtype: float64

### Thống kê nhiều chỉ số

Pandas cung cấp hàm `agg()` cho phép bạn xác định kế hoạch thống kê cho dữ liệu của mình. Với hàm `agg()` (viết tắt của aggregate), bạn có thể thống kê nhiều chỉ số cho cùng một cột. Ví dụ:

In [4]:
d["weight"].agg(["mean", "median"])

mean      55.688272
median    55.000000
Name: weight, dtype: float64

Kết quả trả về sẽ là data frame nếu bạn thống kê cho nhiều cột.

In [5]:
d[["weight", "height"]].agg(["mean", "median"])

,weight,height
mean,55.688272,1.595127
median,55.000000,1.580000


Thông thường chúng ta sẽ hiển thị các trường dữ liệu ở hàng, và các chỉ số thống kê ở cột. Do đó, đối với kết quả như trên, bạn có thể chuyển vị.

In [6]:
d[["weight", "height"]].agg(["mean", "median"]).T

,mean,median
weight,55.688272,55.00
height,1.595127,1.58


Hàm `agg()` rất linh hoạt. Bạn có thể cung cấp hàm tự viết (user-defined function, UDF) cho `agg()`. Các hàm tự viết phải nhận đối số là một series.

In [7]:
def prop(x):
    return x.sum() / x.count() * 100

# Tính tỉ lệ nam giới (sex == 1) trong quần thể nghiên cứu
d["sex"].agg(["count", "sum", prop])

count    330.000000
sum      106.000000
prop      32.121212
Name: sex, dtype: float64

Tên của các phần tử trong series không được phù hợp cho lắm, chúng ta hoàn toàn có thể đổi tên lại. Nếu cần ôn lại thao tác với index và columns, hãy xem lại bài [D02](./02_colindex.ipynb) nhé.

In [8]:
d["sex"].agg(["count", "sum", prop]).rename({"count": "Tổng số", "sum": "Nam (N)", "prop": "Nam (%)"})

Tổng số    330.000000
Nam (N)    106.000000
Nam (%)     32.121212
Name: sex, dtype: float64

Trong trường hợp chỉ yêu cầu một hàm UDF cho `agg()` và nội dung hàm đơn giản, bạn có thể dùng hàm [lambda](../02_inter/02_lambda.ipynb).

In [9]:
# Tạo biến nhị phân: sex == True là nam giới, eso_LA == True là có viêm thực quản
d_compare = d[["sex", "eso_LA"]].gt(0).mask(d[["sex", "eso_LA"]].isna())

# Tính tỉ lệ nam giới và có viêm thực quản
d_compare.agg(lambda x: x.sum() / x.count() * 100)

sex       32.121212
eso_LA    37.049180
dtype: float64

Hàm `agg()` còn có thể tính toán các chỉ số khác nhau cho các cột khác nhau bằng cách cung cấp một từ điển trong đó chìa khóa là tên các cột.

In [10]:
d[["sex", "weight"]].agg({"sex": ["count", "sum", prop], "weight": ["count", "mean", "std"]}).T

,count,sum,prop,mean,std
sex,330.0,106.0,32.121212,NaN,NaN
weight,324.0,NaN,NaN,55.688272,8.606556


### Thêm đối số cho hàm `agg()`

Đôi khi hàm thống kê của bạn cần có thêm đối số. Chẳng hạn, hàm `quantile()` dùng để tính các phân vị.

In [11]:
d["weight"].quantile(q=[0.25, 0.5, 0.75])

0.25    49.0
0.50    55.0
0.75    60.0
Name: weight, dtype: float64

Khi sử dụng hàm này trong `agg()`, bạn có thể cung cấp đối số của hàm `quantile()` cho hàm `agg()`.

In [12]:
d[["weight", "height"]].agg("quantile", q=[0.25, 0.5, 0.75]).T

,0.25,0.50,0.75
weight,49.00,55.00,60.00
height,1.54,1.58,1.65


Bạn chỉ có thể chạy riêng hàm thống kê có đối số. Trong trường hợp có một số dùng đối số, một số không dùng đối số, bạn chạy riêng từng phần rồi ghép chúng lại. Ở điểm này, Pandas không linh hoạt bằng `dplyr` của R.

In [13]:
agg_minmax = d[["weight", "height"]].agg(["min", "max"])
agg_quant = d[["weight", "height"]].agg("quantile", q=[0.25, 0.5, 0.75])
pd.concat([agg_minmax, agg_quant]).T

,min,max,0.25,0.5,0.75
weight,39.00,90.00,49.00,55.00,60.00
height,1.45,1.85,1.54,1.58,1.65


## Phân tầng bằng `groupby()`

Nếu muốn phân tích theo từng dưới nhóm (phân tầng), bạn hãy dùng `groupby()`.

In [14]:
d.groupby("sex")["weight"].mean()

sex
0    52.045662
1    63.285714
Name: weight, dtype: float64

Kết hợp với `agg()` khi có tính toán nhiều chỉ số đồng thời.

In [15]:
d.groupby("sex")["weight"].agg(["mean", "median"])

,mean,median
sex,,
0,52.045662,52.0
1,63.285714,63.0


Bạn có thể làm vậy cho nhiều biến. Lúc này các cột của data frame sẽ là kiểu index đa cấp.

In [16]:
d.groupby("sex")[["weight", "height"]].agg(["mean", "median"])

weight           height       
          mean median      mean median
sex                                   
0    52.045662   52.0  1.557678   1.56
1    63.285714   63.0  1.670381   1.67

Bạn có thể nhóm nhiều hơn một nhóm.

In [17]:
# Tính BMI và tạo nhóm
d["bmi"] = d["weight"] / d["height"] ** 2
d["bmi_group"] = (1 + d["bmi"].ge(18.5) + d["bmi"].gt(23)).mask(d["bmi"].isna()).replace({1: "Thiếu cân", 2: "Bình thường", 3: "Thừa cân"})

# Thống kê theo giới và BMI
d.groupby(["sex", "bmi_group"])[["les_baserestp", "les_irp4s"]].agg(["mean", "std"])

les_baserestp            les_irp4s          
                         mean        std      mean       std
sex bmi_group                                               
0   Bình thường     18.306061   8.013279  6.133333  4.580621
    Thiếu cân       22.770588  14.377837  7.500000  5.662486
    Thừa cân        18.349123   6.928232  5.387719  3.912438
1   Bình thường     16.128846  10.649171  5.319231  4.465337
    Thiếu cân       16.700000   9.700687  4.071429  4.452233
    Thừa cân        12.913043   6.725791  3.839130  4.051473

### Thống kê nhanh bằng `describe()`

Trong trường hợp muốn thống kê nhanh, bạn có thể sử dụng hàm `describe()`.

In [18]:
d[["les_baserestp", "les_irp4s"]].describe()

,les_baserestp,les_irp4s
count,328.000000,328.000000
mean,17.488110,5.602134
std,9.000398,4.525331
min,1.500000,-4.700000
25%,11.000000,2.500000
50%,16.250000,4.900000
75%,22.125000,7.300000
max,66.300000,26.000000


Hàm này vẫn sử dụng được sau khi đã nhóm theo giới và BMI. Đây là cách nhanh gọn trong trường hợp bạn cần quan sát nhanh đặc điểm của số liệu. Trong trường hợp cần tùy biến phức tạp hơn, bạn vẫn phải sử dụng hàm `agg()`.

In [19]:
d.groupby(["sex", "bmi_group"])[["les_baserestp", "les_irp4s"]].describe()

les_baserestp                                           \
                        count       mean        std  min     25%   50%   
sex bmi_group                                                            
0   Bình thường         132.0  18.306061   8.013279  3.7  12.550  18.0   
    Thiếu cân            17.0  22.770588  14.377837  5.5  13.300  17.6   
    Thừa cân             57.0  18.349123   6.928232  7.7  13.300  17.4   
1   Bình thường          52.0  16.128846  10.649171  1.5   9.875  14.8   
    Thiếu cân             7.0  16.700000   9.700687  6.8  10.500  13.6   
    Thừa cân             46.0  12.913043   6.725791  2.0   7.700  11.9   

                              les_irp4s                                        \
                    75%   max     count      mean       std  min    25%   50%   
sex bmi_group                                                                   
0   Bình thường  23.175  46.6     132.0  6.133333  4.580621 -3.0  3.500  5.65   
    Thiếu cân    33.700  51.3      17.0  7.500000  5.662486 -0.6  2.600  6.30   
    Thừa cân     23.300  33.3      57.0  5.387719  3.912438 -0.9  2.700  4.50   
1   Bình thường  19.650  66.3      52.0  5.319231  4.465337 -1.2  2.475  5.10   
    Thiếu cân    20.000  35.5       7.0  4.071429  4.452233 -4.7  2.950  6.10   
    Thừa cân     16.725  28.1      46.0  3.839130  4.051473 -1.5  1.525  3.20   

                              
                   75%   max  
sex bmi_group                 
0   Bình thường  7.225  22.6  
    Thiếu cân    9.000  19.5  
    Thừa cân     7.300  15.1  
1   Bình thường  7.550  26.0  
    Thiếu cân    6.550   8.1  
    Thừa cân     5.350  23.8

---

[Bài trước](./05_na.ipynb) - [Danh sách bài](../README.md) - [Bài sau](./07_apply.ipynb)